In [1]:
!pip install pandas
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 896.2 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
!pip install requests


In [8]:
import requests
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv')


In [9]:
xf=df.head(400)

In [10]:
import os
import openai
import json

os.environ["OPENAI_API_KEY"] = "4b17cb97a16043e59963870587f1d028"
openai.api_type = "azure"
openai.api_base = "https://javelinpreview.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Example review from your dataset
review_text = "Please analyze the sentiment of the following text as positive,negative or neutral(only one word): 'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only has got all the polari but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well do.'"

message_text = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": review_text}
]


response = openai.ChatCompletion.create(
  engine="gpt35",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

sentiment_response = response.choices[0].message.content
print("Review: ",review_text)
print("Sentiment: ",sentiment_response)

Review:  Please analyze the sentiment of the following text as positive,negative or neutral(only one word): 'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only has got all the polari but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of the

In [13]:
import time
def analyze_sentiment(review_text):
    message_text = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": review_text}
    ]

    response = openai.ChatCompletion.create(
        engine="gpt35",
        messages=message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message['content']

# Iterate over the DataFrame and analyze each review

# Add the sentiments back to the DataFrame
sentiments=[]
batch_size = 9
sleep_time = 60  # seconds

for start_index in range(0, len(xf), batch_size):
    end_index = min(start_index + batch_size, len(xf))
    batch = xf[start_index:end_index]

    for index, row in batch.iterrows():
        sentiment = analyze_sentiment("Please analyze the sentiment of the following text as positive,negative or neutral(only one word):" + row['review'])
        sentiments.append(sentiment)
        # Store or print your sentiment analysis result here
        print(f"Processed review {index + 1}: {sentiment}")

    # Wait for 45 seconds before processing the next batch
    if end_index < len(df):  # Prevent sleeping after the last batch
        print(f"Waiting for {sleep_time} seconds before processing the next batch...")
        time.sleep(sleep_time)

xf['sentiment_analysis'] = sentiments



Processed review 1: Negative.
Processed review 2: Positive.
Processed review 3: Positive.
Processed review 4: Negative.
Processed review 5: Positive.
Processed review 6: Positive.
Processed review 7: Positive.
Processed review 8: Negative.
Processed review 9: Negative.
Waiting for 60 seconds before processing the next batch...
Processed review 10: Positive.
Processed review 11: The sentiment of the text is negative.
Processed review 12: negative
Processed review 13: Negative.
Processed review 14: Negative.
Processed review 15: Positive.
Processed review 16: Negative.
Processed review 17: Negative.
Processed review 18: Negative.
Waiting for 60 seconds before processing the next batch...
Processed review 19: Neutral.
Processed review 20: Negative.
Processed review 21: Positive.
Processed review 22: Negative.
Processed review 23: positive
Processed review 24: Negative.
Processed review 25: Negative.
Processed review 26: Neutral.
Processed review 27: positive
Waiting for 60 seconds before 

InvalidRequestError: The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766

In [19]:
xf=xf.head(167)
xf['sentiment_analysis'] = sentiments

<ipython-input-19-df5b6c4a678e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xf['sentiment_analysis'] = sentiments


In [20]:
xf

,review,sentiment,sentiment_analysis
0,One of the other reviewers has mentioned that ...,positive,Negative.
1,A wonderful little production. <br /><br />The...,positive,Positive.
2,I thought this was a wonderful way to spend ti...,positive,Positive.
3,Basically there's a family where a little boy ...,negative,Negative.
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Positive.
...,...,...,...
162,I can't believe it that was the worst movie i ...,negative,Negative
163,"This is one of a rarity of movies, where inste...",negative,Negative.
164,"Even though I'm quite young, The Beatles are m...",positive,Positive.
165,An American Werewolf in London had some funny ...,negative,Negative.


In [21]:
df = xf
df['sentiment'] = df['sentiment'].str.lower().str.replace(r"[^\w\s]", '').str.strip()
df['sentiment_analysis'] = df['sentiment_analysis'].str.lower().str.replace(r"[^\w\s]", '').str.strip()
df

<ipython-input-21-a81c4e38c257>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentiment'] = df['sentiment'].str.lower().str.replace(r"[^\w\s]", '').str.strip()
<ipython-input-21-a81c4e38c257>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].str.lower().str.replace(r"[^\w\s]", '').str.strip()
<ipython-input-21-a81c4e38c257>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentiment_analysis'] = df['sentiment_analysis'].str.lower().str.replace(r"[^\w\s]", '').str.strip()
<ipython-input-21-a81c4e38c257>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,review,sentiment,sentiment_analysis
0,One of the other reviewers has mentioned that ...,positive,negative
1,A wonderful little production. <br /><br />The...,positive,positive
2,I thought this was a wonderful way to spend ti...,positive,positive
3,Basically there's a family where a little boy ...,negative,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,positive
...,...,...,...
162,I can't believe it that was the worst movie i ...,negative,negative
163,"This is one of a rarity of movies, where inste...",negative,negative
164,"Even though I'm quite young, The Beatles are m...",positive,positive
165,An American Werewolf in London had some funny ...,negative,negative


In [22]:
df

,review,sentiment,sentiment_analysis
0,One of the other reviewers has mentioned that ...,positive,negative
1,A wonderful little production. <br /><br />The...,positive,positive
2,I thought this was a wonderful way to spend ti...,positive,positive
3,Basically there's a family where a little boy ...,negative,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,positive
...,...,...,...
162,I can't believe it that was the worst movie i ...,negative,negative
163,"This is one of a rarity of movies, where inste...",negative,negative
164,"Even though I'm quite young, The Beatles are m...",positive,positive
165,An American Werewolf in London had some funny ...,negative,negative


In [23]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
accuracy = accuracy_score(df['sentiment'], df['sentiment_analysis'])
print(f'Accuracy: {accuracy}')
precision = precision_score(df['sentiment'], df['sentiment_analysis'], average='macro')
recall = recall_score(df['sentiment'], df['sentiment_analysis'], average='macro')
f1 = f1_score(df['sentiment'], df['sentiment_analysis'], average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.8922155688622755
Precision: 0.47862263184843834
Recall: 0.44294666278053046
F1 Score: 0.4595588235294118


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
